# Analyzing OD dock data for the Boston Area

## Includes Boston, Cambridge and Sommerville

### Using geopandas (python) and Folium (Javascript)

### Data sources: US Census Massachussets tracts + BlueBike july/2018 trips (293K trips)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rtree
import matplotlib
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
import shapely
from shapely.geometry import Point
import unicodedata
import pysal
from pysal.esda import mapclassify
import datetime as dt

C:\Users\Davor\Anaconda2\envs\py36\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [2]:
trips = pd.read_csv("./201807-bluebikes-tripdata.csv")
trips.shape

(242891, 15)

In [3]:
#let's drop here some rows to make computation faster
trips2 = trips.iloc[:10000].copy()

#TODO: remove weekend trips (or separate from weekday)

trips2['hour'] = trips2['starttime'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f').hour)

trips_morning   = trips2[(trips2['hour'] >= 0)  & (trips2['hour'] <= 12)]  #morning rush hour (up to 9:59:59)
trips_afternoon = trips2[(trips2['hour'] >= 17) & (trips2['hour'] <= 18)] #afternoon rush hour (up to 18:59:59)


trips_afternoon

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,hour
3184,6874,2018-07-01 17:00:07.8670,2018-07-01 18:54:42.2330,23,Boston City Hall - 28 State St,42.358920,-71.057629,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,1148,Subscriber,1969,0,17
3185,6817,2018-07-01 17:00:23.2480,2018-07-01 18:54:01.1560,23,Boston City Hall - 28 State St,42.358920,-71.057629,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,996,Subscriber,1969,0,17
3186,1627,2018-07-01 17:00:24.5550,2018-07-01 17:27:32.3880,20,Aquarium T Stop - 200 Atlantic Ave,42.359912,-71.051430,177,University Park,42.362648,-71.100061,3182,Subscriber,1983,1,17
3187,243,2018-07-01 17:00:32.2870,2018-07-01 17:04:35.7910,16,Back Bay T Stop - Dartmouth St at Stuart St,42.348074,-71.076570,39,Washington St at Rutland St,42.338515,-71.074041,3295,Subscriber,1988,1,17
3188,1830,2018-07-01 17:00:47.3180,2018-07-01 17:31:17.8280,48,Post Office Square - Pearl St at Milk St,42.356755,-71.055407,116,359 Broadway - Broadway at Fayette Street,42.370803,-71.104412,2274,Subscriber,1972,2,17
3189,2955,2018-07-01 17:00:47.7980,2018-07-01 17:50:03.6310,45,Jersey St. at Boylston St.,42.344681,-71.097853,45,Jersey St. at Boylston St.,42.344681,-71.097853,2430,Subscriber,1962,2,17
3190,281,2018-07-01 17:00:56.6040,2018-07-01 17:05:37.7360,26,Washington St at Waltham St,42.341522,-71.068922,200,Washington St at Melnea Cass Blvd,42.332817,-71.081198,2792,Subscriber,1985,1,17
3191,310,2018-07-01 17:01:01.3220,2018-07-01 17:06:12.1750,26,Washington St at Waltham St,42.341522,-71.068922,222,Troy Boston,42.343749,-71.062256,2303,Subscriber,1968,2,17
3192,1779,2018-07-01 17:01:34.3400,2018-07-01 17:31:13.5250,48,Post Office Square - Pearl St at Milk St,42.356755,-71.055407,116,359 Broadway - Broadway at Fayette Street,42.370803,-71.104412,3078,Subscriber,1969,0,17
3193,887,2018-07-01 17:01:32.1460,2018-07-01 17:16:20.1400,10,B.U. Central - 725 Comm. Ave.,42.350406,-71.108279,52,Newbury St at Hereford St,42.348717,-71.085954,2898,Subscriber,1993,1,17


In [4]:
#First create a GeoSeries of trip origins by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
trip_starts_morning_geo = gpd.GeoSeries(trips_morning.apply(lambda z: Point(z['start station longitude'], z['start station latitude']), 1),crs={'init': 'epsg:4326'})
trip_ends_morning_geo = gpd.GeoSeries(trips_morning.apply(lambda z: Point(z['end station longitude'], z['end station latitude']), 1),crs={'init': 'epsg:4326'})

trip_starts_afternoon_geo = gpd.GeoSeries(trips_afternoon.apply(lambda z: Point(z['start station longitude'], z['start station latitude']), 1),crs={'init': 'epsg:4326'})
trip_ends_afternoon_geo = gpd.GeoSeries(trips_afternoon.apply(lambda z: Point(z['end station longitude'], z['end station latitude']), 1),crs={'init': 'epsg:4326'})


#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
geotrips_start_morning = gpd.GeoDataFrame(trips_morning.drop(['start station longitude', 'start station latitude'], 1), geometry=trip_starts_morning_geo)
geotrips_end_morning = gpd.GeoDataFrame(trips_morning.drop(['end station longitude', 'end station latitude'], 1), geometry=trip_ends_morning_geo)

geotrips_start_afternoon = gpd.GeoDataFrame(trips_afternoon.drop(['start station longitude', 'start station latitude'], 1), geometry=trip_starts_afternoon_geo)
geotrips_end_afternoon = gpd.GeoDataFrame(trips_afternoon.drop(['end station longitude', 'end station latitude'], 1), geometry=trip_ends_afternoon_geo)


geotrips_end_morning.shape

(4562, 15)

In [5]:
#Read tracts shapefile into GeoDataFrame
tracts = gpd.read_file('./cb_2017_25_tract_500k.shp').set_index('GEOID')

#Generate Counts of trip origins and destinations per Census Tract
#Spatially join census tracts to trip origin and destination (after projecting) 
# and then group by Tract GEOID while counting the number of trips

tract_counts_start_morning = gpd.tools.sjoin(geotrips_start_morning.to_crs(tracts.crs), tracts.reset_index()).groupby('GEOID').size()
tract_counts_end_morning = gpd.tools.sjoin(geotrips_end_morning.to_crs(tracts.crs), tracts.reset_index()).groupby('GEOID').size()

tract_counts_start_afternoon = gpd.tools.sjoin(geotrips_start_afternoon.to_crs(tracts.crs), tracts.reset_index()).groupby('GEOID').size()
tract_counts_end_afternoon = gpd.tools.sjoin(geotrips_end_afternoon.to_crs(tracts.crs), tracts.reset_index()).groupby('GEOID').size()

tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,NAME,LSAD,ALAND,AWATER,geometry
GEOID,,,,,,,,,
25001013300,25,001,013300,1400000US25001013300,133,CT,12813862,1665621,"POLYGON ((-70.523281 41.768193, -70.5206609999..."
25003900300,25,003,900300,1400000US25003900300,9003,CT,2747524,21953,"POLYGON ((-73.25282399999999 42.465958, -73.25..."
25003921500,25,003,921500,1400000US25003921500,9215,CT,18120993,340404,"POLYGON ((-73.17618999999999 42.701247, -73.17..."
25005610203,25,005,610203,1400000US25005610203,6102.03,CT,14218546,245820,"POLYGON ((-71.225746 42.018209, -71.217208 42...."
25005631600,25,005,631600,1400000US25005631600,6316,CT,1571017,23449,"POLYGON ((-71.28820499999999 41.936269, -71.28..."


In [6]:
#tracts.loc['25025051200', 'StartTripsPerCensusTract'] = 6 #airport
#tracts.loc['25025070700', 'StartTripsPerCensusTract'] = 5
#tracts.loc['25025090100', 'StartTripsPerCensusTract'] = 4 #Between Dorchester and Franklin Park
#tracts.loc['25017352300', 'StartTripsPerCensusTract'] = 3 #east cambridge
#tracts.loc['25025030400', 'StartTripsPerCensusTract'] = 2 #north end (east)
#tracts.loc['25025030200', 'StartTripsPerCensusTract'] = 1 #north end (west)

In [7]:
tracts['MorningStartTripsPerCensusTract'] = (tract_counts_start_morning/(tracts.geometry.area*10**6)).fillna(0)
tracts['MorningEndTripsPerCensusTract'] = (tract_counts_end_morning/(tracts.geometry.area*10**6)).fillna(0)

tracts['AfternoonStartTripsPerCensusTract'] = (tract_counts_start_afternoon/(tracts.geometry.area*10**6)).fillna(0)
tracts['AfternoonEndTripsPerCensusTract'] = (tract_counts_end_afternoon/(tracts.geometry.area*10**6)).fillna(0)

tracts = tracts.reset_index()

#remove the tracts that have no activity
tracts = tracts.loc[(tracts['MorningStartTripsPerCensusTract'] != 0) | (tracts['MorningEndTripsPerCensusTract'] != 0) |
                    (tracts['AfternoonStartTripsPerCensusTract'] != 0) | (tracts['AfternoonEndTripsPerCensusTract'] != 0)]

tracts

,GEOID,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,NAME,LSAD,ALAND,AWATER,geometry,MorningStartTripsPerCensusTract,MorningEndTripsPerCensusTract,AfternoonStartTripsPerCensusTract,AfternoonEndTripsPerCensusTract
39,25017351000,25,017,351000,1400000US25017351000,3510,CT,668313,0,"POLYGON ((-71.11815899999999 42.388025, -71.11...",0.302966,0.027542,0.013771,0.041314
40,25017354600,25,017,354600,1400000US25017354600,3546,CT,1491809,6670,"POLYGON ((-71.15838699999999 42.394593, -71.15...",0.036717,0.018358,0.012239,0.024478
60,25025010103,25,025,010103,1400000US25025010103,101.03,CT,273305,0,"POLYGON ((-71.110732 42.351819, -71.1089959999...",0.913283,0.608856,0.169127,0.135301
62,25025010801,25,025,010801,1400000US25025010801,108.01,CT,127995,0,"POLYGON ((-71.08159499999999 42.353704, -71.07...",2.699603,2.770645,0.852506,0.852506
66,25025061200,25,025,061200,1400000US25025061200,612,CT,1835769,69804,"POLYGON ((-71.06553 42.336029, -71.06238999999...",0.263899,0.206321,0.023991,0.043184
70,25025120600,25,025,120600,1400000US25025120600,1206,CT,258211,0,"POLYGON ((-71.11166899999999 42.31954, -71.110...",0.492092,0.210896,0.035149,0.175747
84,25017352300,25,017,352300,1400000US25017352300,3523,CT,600904,0,"POLYGON ((-71.089181 42.367065, -71.0882589999...",0.807889,1.204212,0.198161,0.228648
85,25017352800,25,017,352800,1400000US25017352800,3528,CT,162599,0,"POLYGON ((-71.10101 42.369317, -71.099925 42.3...",1.801598,0.844499,0.168900,0.168900
87,25017354400,25,017,354400,1400000US25017354400,3544,CT,295694,0,"POLYGON ((-71.141637 42.38719, -71.13579399999...",0.400177,0.092348,0.030783,0.000000
97,25021400100,25,021,400100,1400000US25021400100,4001,CT,861500,13737,"POLYGON ((-71.11717999999999 42.34314, -71.116...",0.326484,0.347547,0.084254,0.105317


In [8]:
#Blue Bike stations region:
west_limit = -71.166491 
east_limit = -71.006098 
north_limit = 42.406302
south_limit = 42.30 
#Create Boston basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
morning_trip_start_map = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 12)

#Update basemap with choropleth

gdf_wgs84 = tracts.to_crs({'init': 'epsg:4326'})

#ts=[0.0] + ps.esda.mapclassify.Fisher_Jenks(tracts['MorningStartTripsPerCensusTract'], k = 5).bins.tolist()
ts=[0.0] + mapclassify.Fisher_Jenks(tracts['MorningStartTripsPerCensusTract'], k = 5).bins.tolist()

#in the new pysal 2.0 the above function will move to pysal.viz.mapclassify


morning_trip_start_map.choropleth(geo_data = gdf_wgs84.to_json(), data = tracts,
                columns = ['GEOID', 'MorningStartTripsPerCensusTract'], key_on = 'feature.properties.{}'.format('GEOID'),
                fill_color = 'YlOrRd', fill_opacity = 0.6, line_opacity = 0.2,  
                threshold_scale = ts)

#folium.TileLayer('stamentoner').add_to(morning_trip_start_map)

morning_trip_start_map

C:\Users\Davor\Anaconda2\envs\py36\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [9]:
morning_trip_end_map = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 12)
ts=[0.0] + mapclassify.Fisher_Jenks(tracts['MorningEndTripsPerCensusTract'], k = 5).bins.tolist()
morning_trip_end_map.choropleth(geo_data = gdf_wgs84.to_json(), data = tracts,
                columns = ['GEOID', 'MorningEndTripsPerCensusTract'], key_on = 'feature.properties.{}'.format('GEOID'),
                fill_color = 'YlOrRd', fill_opacity = 0.6, line_opacity = 0.2,  
                threshold_scale = ts)
morning_trip_end_map

C:\Users\Davor\Anaconda2\envs\py36\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [10]:
afternoon_trip_start_map = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 12)
ts=[0.0] + mapclassify.Fisher_Jenks(tracts['AfternoonStartTripsPerCensusTract'], k = 5).bins.tolist()
afternoon_trip_start_map.choropleth(geo_data = gdf_wgs84.to_json(), data = tracts,
                columns = ['GEOID', 'AfternoonStartTripsPerCensusTract'], key_on = 'feature.properties.{}'.format('GEOID'),
                fill_color = 'YlOrRd', fill_opacity = 0.6, line_opacity = 0.2,  
                threshold_scale = ts)
afternoon_trip_start_map

C:\Users\Davor\Anaconda2\envs\py36\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [11]:
afternoon_trip_end_map = folium.Map([(north_limit+south_limit)/2, (west_limit+east_limit)/2], zoom_start = 12)
ts=[0.0] + mapclassify.Fisher_Jenks(tracts['AfternoonEndTripsPerCensusTract'], k = 5).bins.tolist()
afternoon_trip_end_map.choropleth(geo_data = gdf_wgs84.to_json(), data = tracts,
                columns = ['GEOID', 'AfternoonEndTripsPerCensusTract'], key_on = 'feature.properties.{}'.format('GEOID'),
                fill_color = 'YlOrRd', fill_opacity = 0.6, line_opacity = 0.2,  
                threshold_scale = ts)
afternoon_trip_end_map

C:\Users\Davor\Anaconda2\envs\py36\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [12]:
def add_point_clusters(mapobj, gdf, when):
    coords = []
    for i, row in gdf.iterrows():
        coords.append([row[ when + ' station latitude'], row[when + ' station longitude']])
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of bike trip locations to this layer
    pt_lyr.add_child(MarkerCluster(locations = coords))
    
    #Add this point layer to the map object
    mapobj.add_child(pt_lyr)
    return mapobj

#Update choropleth with point clusters
morning_trip_start_map = add_point_clusters(morning_trip_start_map, trips_morning, 'start')
morning_trip_end_map = add_point_clusters(morning_trip_end_map, trips_morning, 'end')
afternoon_trip_start_map = add_point_clusters(afternoon_trip_start_map, trips_afternoon, 'start')
afternoon_trip_end_map = add_point_clusters(afternoon_trip_end_map, trips_afternoon, 'end')

morning_trip_start_map

In [13]:
folium.LayerControl().add_to(morning_trip_start_map) #Add layer control to toggle on/off
morning_trip_start_map.save('boston_morning_trip_starts.html') #save HTML

folium.LayerControl().add_to(morning_trip_end_map) #Add layer control to toggle on/off
morning_trip_end_map.save('boston_morning_trip_ends.html') #save HTML

folium.LayerControl().add_to(afternoon_trip_start_map) #Add layer control to toggle on/off
afternoon_trip_start_map.save('boston_afternoon_trip_starts.html') #save HTML

folium.LayerControl().add_to(afternoon_trip_end_map) #Add layer control to toggle on/off
afternoon_trip_end_map.save('boston_afternoon_trip_ends.html') #save HTML



#I prefer to open the map in a different window 'cause it's heavy when you have thousands of points.